In [ ]:
import jCMIP as jc
Clist = jc.readList(('CMIP6listnew'))

from netCDF4 import Dataset
from netCDF4 import num2date
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import cftime
import gsw
from datetime import date, timedelta
import pandas as pd
from scipy.interpolate import interp1d

#### Transport plot (dx,dz) and FS boundary

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP   = 'historical'

for mod in range (len(totmod)):
    outfile = ('/home/users/jordi/section26data/' + totmod[mod] + '_' + EXP + '_' + ENS[mod] + '.nc')
    ncid    = Dataset(outfile,'r')
    lev     = ncid.variables['lev'][:]
    lon     = np.squeeze(ncid.variables['lon'][:,:])
    tp15    = ncid.variables['tp15'][:,:,:]
    ncid.close()
    
    #Find FS boundary
    tp15mean = np.nanmean(tp15,axis=0)
    tprow15 = np.nansum(tp15mean,axis=1)
    mocbtv  = -1*np.cumsum(tprow15[::-1])[::-1]
    imax2   = np.where(mocbtv == max(mocbtv))
    imax1   = imax2[0]
    imax    = imax1[0]
    
    tpcol15 = np.nansum(tp15mean[0:imax,:],axis=0)
    mx     = np.nanargmin(np.abs(tpcol15 - max(tpcol15)))
    idx2   = np.where(tpcol15[mx::]<0)
    idx1   = idx2[0]
    idx = idx1[0] + mx
    
    #if totmod[mod] == 'MPI-ESM1-2-HR':
    #    tpcol15 = np.nansum(tp15mean[0:imax,:],axis=0)
    #    idx2   = np.where(tpcol15[8::]<0)
    #    idx1   = idx2[0]
    #    idx = idx1[0] + 8
    #    print(idx)
    
    plt.figure(mod)
    plt.pcolormesh(lon,lev,tp15mean)
    plt.axvline(x=lon[idx],color='C3')
    plt.axhline(y=lev[imax],color='C3')
    plt.gca().invert_yaxis()
    plt.title(totmod[mod])
    plt.colorbar()

#### Check MPI-ESM1-2-HR

In [ ]:
totmod =  'MPI-ESM1-2-HR'
ENS    =  'r1i1p1f1'
EXP    = 'historical'

outfile = ('/home/users/jordi/section26data/' + totmod + '_' + EXP + '_' + ENS + '.nc')
ncid    = Dataset(outfile,'r')
lev     = ncid.variables['lev'][:]
lon     = np.squeeze(ncid.variables['lon'][:,:])
tp15    = ncid.variables['tp15'][:,:,:]
ncid.close()
    
#Find FS boundary
tp15mean = np.nanmean(tp15,axis=0)
tpcol15  = np.nansum(tp15mean,axis=0)
print(tpcol15)